In [1]:
import tensorflow as tf
import numpy as np
import datetime
import time

import sklearn.metrics

In [16]:
def genObs(length=20):
    return np.random.choice(range(2), size=length)

def genTarget(x):
    return np.sum(x)

def genSample(num, length=20):
    x = [genObs(length=length) for _ in range(num)]
    y = [genTarget(t) for t in x]
    return np.array(x), np.array(y)

def randomBatch(tensorTuple, batchSize=64):
    ids = np.random.choice(range(tensorTuple[0].shape[0]), batchSize)
    return (x[ids,] for x in tensorTuple)

In [7]:
train_x, train_y = genSample(10000)
valid_x, valid_y = genSample(1000)

In [124]:
SEQ_LEN = 20
RNN_SIZE = 5

tf.reset_default_graph()

tfi_x = tf.placeholder(shape=(None, SEQ_LEN), dtype=tf.float32)
tfi_y = tf.placeholder(shape=(None), dtype=tf.int64)

tfX = tf.reshape(tfi_x, shape=(tf.shape(tfi_x)[0], tf.shape(tfi_x)[1], 1))
tfY = tf.one_hot(tfi_y, SEQ_LEN + 1, dtype=tf.float32)

#rnnCell = tf.nn.rnn_cell.LSTMCell(num_units=RNN_SIZE, activation=tf.nn.tanh, state_is_tuple=False)
rnnCell = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.GRUCell(num_units=RNN_SIZE, activation=tf.nn.tanh) for _ in range(3)])

tfH, tfO = tf.nn.dynamic_rnn(rnnCell, inputs=tfX, dtype=tf.float32)

tfOut = tf.layers.dense(tfO[2], SEQ_LEN + 1, activation=tf.nn.relu)

tfLoss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tfY, logits=tfOut))
tfTrain = tf.train.AdamOptimizer(1e-3).minimize(tfLoss)

tfOutP = tf.nn.softmax(tfOut)
tfOutR = tf.argmax(tfOutP, axis=1)

tfAccuracy = tf.reduce_mean(tf.cast(tf.equal(tfi_y, tfOutR), dtype=tf.float32))
print('Graph creation complete')

Graph creation complete


In [125]:
batch_size = 1000
num_steps  = 20
num_epochs = 150

valid_batch = {tfi_x: valid_x, tfi_y: valid_y}
with tf.Session() as tfs:
    tfs.run(tf.global_variables_initializer())
    for i in range(num_epochs):
        mini_x, mini_y = randomBatch((train_x, train_y), batchSize=batch_size)
        train_batch = {tfi_x:mini_x, tfi_y:mini_y}
        l0 = tfLoss.eval(feed_dict=train_batch)
        t0 = time.perf_counter()
        for j in range(num_steps):
            tfTrain.run(feed_dict=train_batch)
        t1 = time.perf_counter()
        l1 = tfLoss.eval(feed_dict=train_batch)
        lv = tfLoss.eval(feed_dict=valid_batch)
        ac = tfAccuracy.eval(feed_dict=valid_batch)
        
        print('Epoch {0} ({1:1.3} sec): loss changed from {2:1.3} to {3:1.3}\t\tVL:{4:1.3}\t\tAC:{5:1.3}'.format(i,t1-t0,l0,l1,lv,ac))
    valid_p, valid_r = tfs.run([tfOutP, tfOutR], feed_dict=valid_batch)

Epoch 0 (0.647 sec): loss changed from 3.07 to 3.04		VL:3.04		AC:0.12
Epoch 1 (0.473 sec): loss changed from 3.04 to 3.03		VL:3.03		AC:0.12
Epoch 2 (0.495 sec): loss changed from 3.03 to 3.01		VL:3.01		AC:0.12
Epoch 3 (0.474 sec): loss changed from 3.01 to 2.99		VL:2.99		AC:0.12
Epoch 4 (0.59 sec): loss changed from 2.99 to 2.96		VL:2.96		AC:0.12
Epoch 5 (0.486 sec): loss changed from 2.98 to 2.97		VL:2.95		AC:0.048
Epoch 6 (0.482 sec): loss changed from 2.94 to 2.93		VL:2.95		AC:0.123
Epoch 7 (0.556 sec): loss changed from 2.94 to 2.94		VL:2.95		AC:0.123
Epoch 8 (0.534 sec): loss changed from 2.95 to 2.95		VL:2.95		AC:0.12
Epoch 9 (0.465 sec): loss changed from 2.96 to 2.96		VL:2.95		AC:0.081
Epoch 10 (0.47 sec): loss changed from 2.95 to 2.95		VL:2.95		AC:0.123
Epoch 11 (0.604 sec): loss changed from 2.94 to 2.93		VL:2.95		AC:0.123
Epoch 12 (0.484 sec): loss changed from 2.96 to 2.95		VL:2.95		AC:0.12
Epoch 13 (0.6 sec): loss changed from 2.93 to 2.93		VL:2.95		AC:0.12
Epoch 14 (0.47

Epoch 115 (0.527 sec): loss changed from 1.83 to 1.83		VL:1.87		AC:0.416
Epoch 116 (0.554 sec): loss changed from 1.83 to 1.83		VL:1.87		AC:0.416
Epoch 117 (0.516 sec): loss changed from 1.86 to 1.85		VL:1.86		AC:0.416
Epoch 118 (0.519 sec): loss changed from 1.79 to 1.78		VL:1.86		AC:0.416
Epoch 119 (0.48 sec): loss changed from 1.84 to 1.84		VL:1.86		AC:0.416
Epoch 120 (0.492 sec): loss changed from 1.78 to 1.77		VL:1.86		AC:0.416
Epoch 121 (0.537 sec): loss changed from 1.82 to 1.81		VL:1.85		AC:0.416
Epoch 122 (0.475 sec): loss changed from 1.82 to 1.82		VL:1.85		AC:0.416
Epoch 123 (0.642 sec): loss changed from 1.85 to 1.85		VL:1.85		AC:0.416
Epoch 124 (0.507 sec): loss changed from 1.84 to 1.83		VL:1.85		AC:0.415
Epoch 125 (0.493 sec): loss changed from 1.81 to 1.81		VL:1.84		AC:0.415
Epoch 126 (0.475 sec): loss changed from 1.75 to 1.74		VL:1.84		AC:0.416
Epoch 127 (0.514 sec): loss changed from 1.81 to 1.8		VL:1.84		AC:0.415
Epoch 128 (0.549 sec): loss changed from 1.78 to 1.77

KeyboardInterrupt: 

In [116]:
valid_y[:15],valid_r[:15]

(array([13,  7, 11,  8, 11,  9, 10,  9, 11, 10, 10, 11,  9, 10,  8]),
 array([13,  7, 11,  8, 11,  9, 10,  9, 11, 10, 10, 11,  9, 10,  8], dtype=int64))

In [121]:
valid_y[valid_y>13], valid_r[valid_y>13]

(array([15, 14, 14, 14, 15, 15, 14, 17, 14, 14, 14, 15, 14, 14, 14, 14, 14,
        15, 14, 14, 14, 14, 15, 15, 16, 14, 14, 16, 18, 14, 15, 15, 14, 14,
        14, 15, 14, 14, 14, 16, 14, 15, 16, 14, 14, 16, 14, 14, 14, 14, 14,
        14, 15, 14, 14, 14, 14, 14, 15, 14, 14, 14, 14, 15, 14, 16, 14, 14,
        14, 15, 16, 14, 15]),
 array([15, 14, 14, 14, 15, 15, 14, 15, 14, 14, 14, 15, 14, 14, 14, 14, 14,
        15, 14, 14, 14, 14, 15, 15, 15, 14, 14, 15, 15, 14, 15, 15, 14, 14,
        14, 15, 14, 14, 14, 15, 14, 15, 15, 14, 14, 15, 14, 14, 14, 14, 14,
        14, 15, 14, 14, 14, 14, 14, 15, 14, 14, 14, 14, 15, 14, 15, 14, 14,
        14, 15, 15, 14, 15], dtype=int64))